# Programme de calcul des dimensions de l'enceinte connectée

## Objectif

Le programme sert à résoudre l'équations non-linéaire décrivant les dimensions de l'enceinte connectée (modèle de Les Frères Poulain sur YouTube : https://youtu.be/ollr-PpCMAw). Pour cela on utilise la méthode de Newton.

## Licence 

Ce programme est sous licence CC-Zéro (licence libre non copyleft). Libre à vous d'utiliser ce code, de le modifier, de le partager ou même de le commercialiser.

## Auteur

Le code est écrit par Valentin Besse (valentinbesse@aumbox.net).

In [17]:
import math # importe le module mathematique.

# Declaration des variables de depart
volumeEnceinte = 8*10**6 # Volume de l'enceinte (mm3)
epaisseurBois = 20 # Epaisseur du bois (mm)
volumeBassReflex = 2*math.pi*70*115 # Volume du Bass Reflex à retiré (mm3)
volumeCorrige = volumeEnceinte + volumeBassReflex # Volume corrigé de l'intérieur de la boite.
ratioA = 2.6 # Ratio entre A et B
ratioC = 1.6 # Ratio entre C et B

# Definition des fonctions nécessaires au calcul de la méthode de Newton.

# f(x) = 0
def dx(f, x):
    return abs(0-f(x))

# Methode de Newton
def newtons_method(f, df, x0, e):
    delta = dx(f, x0)
    while delta > e:
        x0 = x0 - f(x0)/df(x0)
        delta = dx(f, x0)
    print('La largeur A est : %1.0f' % (ratioA*x0), 'mm') # Largeur
    print('La profondeur B est : %1.0f' % x0, 'mm') # Profondeur
    print('La hauteur C est : %1.0f' % (ratioC*x0), 'mm') # Hauteur
    print('La largeur intérieure a est : %1.0f' % (ratioA*x0 - 2*epaisseurBois), 'mm') # Largeur intérieure
    print('La profondeur intérieure b est : %1.0f' % (x0 - 2*epaisseurBois), 'mm') # Profondeur intérieure
    print('La hauteur intérieure c est : %1.0f' % (ratioC*x0 - 2*epaisseurBois), 'mm') # Hauteur intérieure
    
    
# Fonction f(x)
def f(x):
    return volumeCorrige - 8/9*(x - 2*epaisseurBois)*(1.6*x - 2*epaisseurBois)*(2.6*x - 2*epaisseurBois)

# Derivee de la fonction f(x)
def df(x):
    return -8/9*((2.6*x - 2*epaisseurBois)*((1.6*x - 2*epaisseurBois) + 1.6*(x - 2*epaisseurBois)) + 2.6*(x - 2*epaisseurBois)*(1.6*x - 2*epaisseurBois))

x0s = [100]
for x0 in x0s:
     newtons_method(f, df, x0, 1e-5)

La largeur A est : 408 mm
La profondeur B est : 157 mm
La hauteur C est : 251 mm
La largeur intérieure a est : 368 mm
La profondeur intérieure b est : 117 mm
La hauteur intérieure c est : 211 mm
